# 学习PyTorch-Day3

In [4]:
import torch
import numpy as np

# for mac
device = torch.device("mps")

# for cuda
# device = torch.device("cuda")

## Transforms-变换

数据并不总是按照机器学习算法所需的形式出现，我们经常需要对数据执行变换，使其适合训练

In [5]:
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(root="data", 
                           train=True,
                           download=False,
                           transform=ToTensor())

### ToTensor()

ToTensor可以将一张图像或者Numpy数组转换成 __FloatTensor__ ，并将图像的像素强度值缩放在[0., 1.] 范围内

### Lambda变换

Lambda可以封装任何自定义的变换函数

## 构建神经网络

神经网络由各种层/模块构成，这些层/模块包装了对数据的操作
- __torch.nn__ 中包括了构建神经网络所需的各种模板，PyTorch模板中的每个模块都是 __nn.Module__ 的子集

### 实例：构建分类网络，对FashionMNIST数据集进行分类

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 指认设备
device = torch.device(
    "cuda" 
    if torch.cuda.is_available()
    else "mps" 
    if torch.backends.mps.is_available()
    else "cpu"
)

print(device)

mps


- 定义神经网络（继承于 __nn.Module__）

In [7]:
from turtle import forward


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 展平处理
        self.flatten = nn.Flatten()
        # 全连接+激活
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

- 实例化网络，指认到设备上

In [8]:
model = NeuralNetwork()
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

- 初始化输入
- 因为此问题是互斥分类问题，即结果只可能属于一类，所以应使用 Softmax 激活函数
- nn.Softmax(dim=1)，此处的 dim 表示在哪个维度上 和为1，此处应该是10类输出之和1，所以应为 dim=1
- Tensor.argmax() == torch.argmax(Tensor, dim)，返回输入Tensor中最大值的下标

In [9]:
t = torch.rand(1, 28, 28, device=device)
out = model(t)
# 因为此问题是互斥分类问题，即结果只可能属于一类，所以应使用 Softmax 激活函数
softmax_func = nn.Softmax(dim=1)  # 此处的 dim 表示在哪个维度上 和为1，此处应该是10类输出之和1，所以应为 dim=1
pred_prob = softmax_func(out)
# argmax
pred_cls = torch.argmax(pred_prob, dim=1)
print(pred_cls.item())

2


### 具体分析

In [10]:
# 以一张图像为例
input_image = torch.rand(3, 28, 28)
# shape是Tensor的属性，size()是Tensor的方法，二者均可以获得Tensor的维度
print(input_image.shape)
print(input_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 28, 28])


- __nn.Flatten__

可以将 2D图像展平，变成一个拥有 784个像素（连续）的数组

In [11]:
from torch import nn

flatten = nn.Flatten()

flat_img = flatten(input_image)
print(flat_img.shape)

torch.Size([3, 784])


- __nn.Linear__

线性层是一个使用其存储的权重和偏差对输入应用线性变换的模块

In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden = layer1(flat_img)
print(hidden.shape)

torch.Size([3, 20])


- __nn.ReLU__

非线性激活函数在输入和输出之间建立复杂的映射关系

它们在线性变换后应用以引入非线性，帮助神经网络学习各种现象

In [13]:
print("Before ReLU: {}".format(hidden))
hidden1 = nn.ReLU(inplace=True)(hidden)
print("After ReLU: {}".format(hidden1))

Before ReLU: tensor([[-0.1219, -0.3522, -0.7829,  0.2842, -0.1779, -0.4762, -0.6249,  0.0191,
          0.0280,  0.4442, -0.1072, -0.0033, -0.1826, -0.6254,  0.1005, -0.0075,
         -0.3307, -0.0596, -0.2938,  0.2695],
        [-0.3704, -0.4180, -0.4482,  0.1459, -0.4321,  0.0838, -0.6310, -0.0928,
          0.2291,  0.2921, -0.1476,  0.0014, -0.2576, -0.7668,  0.0605, -0.4167,
         -0.2690,  0.1728, -0.0496,  0.4177],
        [-0.0715, -0.3375, -0.6984, -0.1599, -0.1517, -0.2152, -0.8183, -0.0715,
          0.1835,  0.2847, -0.2910,  0.0037, -0.3580, -0.7739,  0.2845, -0.3604,
         -0.0760, -0.1074, -0.4091,  0.5348]], grad_fn=<AddmmBackward0>)
After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.2842, 0.0000, 0.0000, 0.0000, 0.0191, 0.0280,
         0.4442, 0.0000, 0.0000, 0.0000, 0.0000, 0.1005, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2695],
        [0.0000, 0.0000, 0.0000, 0.1459, 0.0000, 0.0838, 0.0000, 0.0000, 0.2291,
         0.2921, 0.0000, 0.0014, 0.0000, 0.0000, 0.0605

- __nn.Sequential__

nn.Sequential是存放模块的有序容器，数据按照模块被定义的顺序经过各个模块

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(inplace=True),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
result = seq_modules(input_image)
print(result.shape)

torch.Size([3, 10])


- __nn.Softmax__

网络最后一层的返回值，送入 __nn.Softmax__ 模块，值被缩放到 [0, 1]，表示模型对每个类别的预测概率

In [15]:
softmax = nn.Softmax(dim=1)  # dim=1表示对输入Tensor的dim=1维度进行Softmax处理，sum = 1

In [16]:
pred_prob = softmax(result)
print(pred_prob)

tensor([[0.1133, 0.0926, 0.0981, 0.0967, 0.0957, 0.1005, 0.1000, 0.0851, 0.1183,
         0.0997],
        [0.1286, 0.1006, 0.0962, 0.0928, 0.0914, 0.1084, 0.0974, 0.0858, 0.1067,
         0.0921],
        [0.1126, 0.0960, 0.1099, 0.0994, 0.0970, 0.1117, 0.0957, 0.0864, 0.1075,
         0.0836]], grad_fn=<SoftmaxBackward0>)


### 模型参数

- 遍历模型参数

In [19]:
print("Model Architecture: {}".format(model))

Model Architecture: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [20]:
for name, parameter in model.named_parameters():
    print("name: {}, parameter: {}".format(name, parameter))

/opt/anaconda3/envs/ml/lib/python3.7/site-packages/torch/_tensor_str.py:103: UserWarning: The operator 'aten::bitwise_and.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484744261/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(tensor_view, torch.isfinite(tensor_view) & tensor_view.ne(0))


name: linear_relu_stack.0.weight, parameter: Parameter containing:
tensor([[ 1.6962e-02,  3.2612e-02, -6.2463e-03,  ...,  2.8459e-03,
          7.1751e-03,  2.4258e-02],
        [-8.0782e-03, -2.1115e-02, -1.3490e-02,  ..., -1.0968e-02,
          3.4302e-02, -1.7684e-02],
        [ 9.9019e-03, -5.1077e-05, -3.5250e-02,  ...,  6.1434e-03,
         -2.5816e-04, -2.1763e-02],
        ...,
        [ 1.2548e-02, -2.8348e-02,  3.1291e-02,  ...,  8.0662e-03,
          8.3008e-03, -1.7072e-02],
        [-2.6389e-02,  2.0579e-02,  2.3972e-02,  ..., -2.6682e-02,
         -1.8750e-02, -2.6942e-02],
        [-1.5087e-02, -8.9340e-03, -2.8124e-02,  ..., -1.1601e-02,
         -3.4300e-02, -3.4701e-02]], device='mps:0', requires_grad=True)
name: linear_relu_stack.0.bias, parameter: Parameter containing:
tensor([ 3.2329e-02,  1.9988e-02, -1.5272e-02,  6.3280e-03,  1.1474e-02,
         1.2977e-02, -1.8993e-03,  1.6732e-02,  9.3199e-05,  3.0604e-02,
         3.1565e-02, -2.0835e-02, -1.6370e-02,  3.4749